# Triangulation analysis

In [ ]:
print(sessionInfo())
version
gc()

## Load packages

In [ ]:
# Load required R packages
library(tidyverse)
library(igraph)
library(ggraph)
library(scales)
library(ggplot2)

source("Functions/triangulation_distances.R")
source("Functions/run_triangulation_distances.R")

## Define Input/Output paths 
Define an input and output path. The input should be a csv file that contains at least x and y coordinates, a column, indicating the region, cell types or any other kind of group and if possible a comparisson.

In [ ]:
input_path <- "/path_to_input.csv"
# Output
output_path <- "/output_path"
dir.create(output_path)

### Triangulation analysis
#### Optional: Load already calculated triangulations

In [ ]:
# Skip if calculated yet
load_calculated_triangulation_data <- "no" # type "yes" or "no"

triangulation_distances_path <- "/observed_distances"
iterated_triangulation_distances_path <- "/expected_distances"

#### Specify parameters

In [ ]:
# Define columns of input data
x_position_column <- "x"
y_position_column <- "y"
cell_type_column <- "subcluster8"
region_column <- "unique_region"
compare_condition_column <- "Nodal_disease"

if (compare_condition_column != NA) {
  condition_1 <- "Treated"
  condition_2 <- "Untreated"

  output_path <- paste0(output_path, "/", compare_condition_column)
  dir.create(output_path)
}

# Set distance threshold for observed cell-cell interactions
# distance_threshold = 128  corresponds to 100um
distance_threshold <- 128

#### Read data for triangulation 

In [ ]:
# Read data
df <- read.csv(codex_sc_path, header = TRUE)

if (load_calculated_triangulation_data == "yes") {
  triangulation_distances <- read_csv(triangulation_distances_path)
  iterated_triangulation_distances <- read_csv(iterated_triangulation_distances_path)
}

colnames(df)[1] <- "index"
cell_index_column <- "index"

if (compare_condition_column != NA) {
  df <- df[, c(cell_index_column, x_position_column, y_position_column, compare_condition_column, region_column, cell_type_column)]

  metadata <- df %>%
    dplyr::select(compare_condition_column, region_column) %>%
    dplyr::distinct(.keep_all = TRUE)
} else {
  df <- df[, c(cell_index_column, x_position_column, y_position_column, region_column, cell_type_column)]

  metadata <- df %>%
    dplyr::select(region_column) %>%
    dplyr::distinct(.keep_all = TRUE)
}

#### Run Trianagulation 

In [ ]:
run_triangulation_distances <- function(df = df,
                                        cell_index_column = cell_index_column,
                                        x_position_column = x_position_column,
                                        y_position_column = y_position_column,
                                        cell_type_column = cell_type_column,
                                        region_column = region_column,
                                        calc_avg_distance = TRUE,
                                        output_path = output_path,
                                        number_of_iterations = 100,
                                        num_cores = NULL)

#### Further analysis triangulation - modified

In [ ]:
if (compare_condition_column != NA) {
  names(metadata)[names(metadata) == compare_condition_column] <- "comparison"
  # Reformat observed dataset
  observed_distances <- triangulation_distances %>%
    # Append metadata
    dplyr::left_join(metadata,
      by = c("unique_region")
    ) %>%
    dplyr::filter(distance <= distance_threshold) %>%
    # Calculate the average distance to every cell type for each cell
    dplyr::group_by(celltype1_index, celltype1, celltype2, comparison, unique_region) %>%
    dplyr::summarize(mean_per_cell = mean(distance)) %>%
    dplyr::ungroup() %>%
    # Calculate the average distance between cell type to cell type on a per group basis
    dplyr::group_by(celltype1, celltype2, comparison) %>%
    dplyr::summarize(
      observed = list(mean_per_cell),
      observed_mean = mean(unlist(observed), na.rm = TRUE)
    ) %>%
    dplyr::ungroup()

  # Reformat exepcted dataset
  expected_distances <- iterated_triangulation_distances %>%
    # Append metadata
    dplyr::left_join(metadata,
      by = c("unique_region")
    ) %>%
    dplyr::filter(mean_dist <= distance_threshold) %>%
    # Calculate expected mean distance and list values
    dplyr::group_by(celltype1, celltype2, comparison) %>%
    dplyr::summarize(
      expected = list(mean_dist),
      expected_mean = mean(mean_dist, na.rm = TRUE)
    ) %>%
    dplyr::ungroup()

  # drop comparisons with low numbers of observations
  # This step was implemented to reduce the influence of rare cell types - usually, these tend to dominate statistics as small changes are already highly significant
  logical_list_observed <- list()
  for (i in 1:nrow(observed_distances)) {
    print(i)
    vec <- observed_distances[i, "observed"]

    if (length(unlist(vec)) > 10) {
      logical_list_observed[[as.character(i)]] <- TRUE
    } else {
      logical_list_observed[[as.character(i)]] <- FALSE
    }
  }

  list_observed <- unlist(logical_list_observed)

  observed_distances$keep <- list_observed

  observed_distances <- observed_distances %>% filter(keep == TRUE)

  # Perform the same filtering on the expected distances
  # drop comparisons with low numbers of observations
  # This step was implemented to reduce the influence of rare cell types - usually, these tend to dominate statistics as small changes are already highly significant
  logical_list_expected <- list()
  for (i in 1:nrow(expected_distances)) {
    print(i)
    vec <- expected_distances[i, "expected"]

    if (length(unlist(vec)) > 10) {
      logical_list_expected[[as.character(i)]] <- TRUE
    } else {
      logical_list_expected[[as.character(i)]] <- FALSE
    }
  }

  list_observed <- unlist(logical_list_expected)

  expected_distances$keep <- list_observed

  expected_distances <- expected_distances %>% filter(keep == TRUE)

  # Calculate pvalues and log fold differences
  distance_pvals <- expected_distances %>%
    dplyr::left_join(observed_distances,
      by = c("celltype1", "celltype2", "comparison")
    ) %>%
    # Calculate wilcoxon test between observed and expected distances
    dplyr::group_by(celltype1, celltype2, comparison) %>%
    dplyr::mutate(pvalue = wilcox.test(unlist(expected), unlist(observed), exact = FALSE)$p.value) %>%
    dplyr::ungroup() %>%
    dplyr::select(-observed, -expected) %>%
    # Calculate log fold enrichment
    dplyr::mutate(
      logfold_group = log2(observed_mean / expected_mean),
      interaction = paste0(celltype1, " --> ", celltype2)
    )

  # Get order of plot by magnitude of logfold differences between groups
  intermed <- distance_pvals %>%
    dplyr::select(interaction, comparison, logfold_group) %>%
    tidyr::spread(key = comparison, value = logfold_group)

  intermed$difference <- (intermed[, condition_2] - intermed[, condition_1])

  ord <- (intermed %>%
    dplyr::filter(!is.na(difference)) %>%
    dplyr::arrange(condition_1))$interaction

  # Assign interaction order
  distance_pvals$interaction <- factor(distance_pvals$interaction,
    levels = ord
  )

  distance_pvals <- write_csv(distance_pvals, paste0(specific_output, "/distance_pvals.csv"))

  # general filtering before analysis of the results
  distance_pvals_sig <- distance_pvals %>%
    filter(pvalue < 0.05) %>% # keep only significant results
    # filter(celltype1 != celltype2) %>% # compare only different cell types
    filter(!is.na(observed_mean)) %>% # ignore columns without observation
    filter(celltype1 != "unknown") %>% # drop cells of type unknown
    filter(celltype2 != "unknown") %>%
    filter(celltype1 != "noise") %>% # drop cells of type noise
    filter(celltype2 != "noise") %>%
    filter(!is.na(comparison))
} else {
  # Reformat observed dataset
  observed_distances <- triangulation_distances %>%
    # Append metadata
    dplyr::left_join(metadata,
      by = c("unique_region")
    ) %>%
    dplyr::filter(distance <= distance_threshold) %>%
    # Calculate the average distance to every cell type for each cell
    dplyr::group_by(celltype1_index, celltype1, celltype2, unique_region) %>%
    dplyr::summarize(mean_per_cell = mean(distance)) %>%
    dplyr::ungroup() %>%
    # Calculate the average distance between cell type to cell type on a per group basis
    dplyr::group_by(celltype1, celltype2) %>%
    dplyr::summarize(
      observed = list(mean_per_cell),
      observed_mean = mean(unlist(observed), na.rm = TRUE)
    ) %>%
    dplyr::ungroup()

  # Reformat exepcted dataset
  expected_distances <- iterated_triangulation_distances %>%
    # Append metadata
    dplyr::left_join(metadata,
      by = c("unique_region")
    ) %>%
    dplyr::filter(mean_dist <= distance_threshold) %>%
    # Calculate expected mean distance and list values
    dplyr::group_by(celltype1, celltype2) %>%
    dplyr::summarize(
      expected = list(mean_dist),
      expected_mean = mean(mean_dist, na.rm = TRUE)
    ) %>%
    dplyr::ungroup()

  # drop comparisons with low numbers of observations
  # This step was implemented to reduce the influence of rare cell types - usually, these tend to dominate statistics as small changes are already highly significant
  logical_list_observed <- list()
  for (i in 1:nrow(observed_distances)) {
    print(i)
    vec <- observed_distances[i, "observed"]

    if (length(unlist(vec)) > 10) {
      logical_list_observed[[as.character(i)]] <- TRUE
    } else {
      logical_list_observed[[as.character(i)]] <- FALSE
    }
  }

  list_observed <- unlist(logical_list_observed)

  observed_distances$keep <- list_observed

  observed_distances <- observed_distances %>% filter(keep == TRUE)

  # Perform the same filtering on the expected distances
  # drop comparisons with low numbers of observations
  # This step was implemented to reduce the influence of rare cell types - usually, these tend to dominate statistics as small changes are already highly significant
  logical_list_expected <- list()
  for (i in 1:nrow(expected_distances)) {
    print(i)
    vec <- expected_distances[i, "expected"]

    if (length(unlist(vec)) > 10) {
      logical_list_expected[[as.character(i)]] <- TRUE
    } else {
      logical_list_expected[[as.character(i)]] <- FALSE
    }
  }

  list_observed <- unlist(logical_list_expected)

  expected_distances$keep <- list_observed

  expected_distances <- expected_distances %>% filter(keep == TRUE)

  # Calculate pvalues and log fold differences
  distance_pvals <- expected_distances %>%
    dplyr::left_join(observed_distances,
      by = c("celltype1", "celltype2")
    ) %>%
    # Calculate wilcoxon test between observed and expected distances
    dplyr::group_by(celltype1, celltype2) %>%
    dplyr::mutate(pvalue = wilcox.test(unlist(expected), unlist(observed), exact = FALSE)$p.value) %>%
    dplyr::ungroup() %>%
    dplyr::select(-observed, -expected) %>%
    # Calculate log fold enrichment
    dplyr::mutate(
      logfold_group = log2(observed_mean / expected_mean),
      interaction = paste0(celltype1, " --> ", celltype2)
    )

  distance_pvals$interaction <- factor(distance_pvals$interaction,
    levels = ord
  )

  distance_pvals <- write_csv(distance_pvals, paste0(output_dir, "/distance_pvals.csv"))

  # general filtering before analysis of the results
  distance_pvals_sig <- distance_pvals %>%
    filter(pvalue < 0.05) %>% # keep only significant results
    # filter(celltype1 != celltype2) %>% # compare only different cell types
    filter(!is.na(observed_mean)) %>% # ignore columns without observation
    filter(celltype1 != "unknown") %>% # drop cells of type unknown
    filter(celltype2 != "unknown") %>%
    filter(celltype1 != "noise") %>% # drop cells of type noise
    filter(celltype2 != "noise")
}


Subset list for most interesting interactions

In [ ]:
# Check if compare_condition_column is not NA
if (!is.na(compare_condition_column)) {
  # Filter distance_pvals_sig for non-NA interaction and logfold_group values
  distance_pvals_sig <- distance_pvals_sig %>%
    filter(!is.na(interaction)) %>%
    filter(!is.na(logfold_group))
  
  # Calculate absolute logfold values
  distance_pvals_sig$abs_logfold <- abs(distance_pvals_sig$logfold_group)
  
  # Filter distance_pvals_sig based on absolute logfold threshold
  distance_pvals_sig_filt <- distance_pvals_sig %>% filter(abs_logfold >= 0.1)
  
  # Remove duplicated interactions based on sorted celltype1 and celltype2 columns
  distance_pvals_sig_filt <- distance_pvals_sig_filt[!duplicated(t(apply(distance_pvals_sig_filt[c("celltype1", "celltype2")], 1, sort))), ]
  
  # Filter distance_pvals based on the interesting interactions
  distance_pvals_interesting <- distance_pvals[distance_pvals$interaction %in% distance_pvals_sig_filt$interaction, ]
  
  # Filter distance_pvals_interesting for non-NA treatment values
  distance_pvals_interesting <- distance_pvals_interesting %>% filter(!is.na(treatment))
  
  # Group distance_pvals_interesting by interaction for condition_1 (positive treatment)
  df_Rpos <- distance_pvals_interesting %>%
    filter(treatment == condition_1) %>%
    group_by(interaction)
  
  # Group distance_pvals_interesting by interaction for condition_2 (negative treatment)
  df_Rneg <- distance_pvals_interesting %>%
    filter(treatment == condition_2) %>%
    group_by(interaction)
  
  # Set row names as interaction values for df_Rpos and df_Rneg
  rownames(df_Rpos) <- df_Rpos$interaction
  rownames(df_Rneg) <- df_Rneg$interaction
  
  # Select relevant columns from df_Rneg and rename them
  df_Rneg <- df_Rneg[, c("logfold_group", "interaction")]
  colnames(df_Rneg) <- c("logfold_group_Rneg", "interaction_Rneg")
  
  # Combine df_Rpos and df_Rneg into a single data frame
  comb <- cbind(df_Rpos, df_Rneg)
  
  # Calculate the absolute difference between logfold_group and logfold_group_Rneg
  comb$difference_R <- abs(comb$logfold_group - comb$logfold_group_Rneg)
  
  # Filter comb based on difference threshold
  comb_filt <- comb %>% filter(difference_R >= 0.2)
  
  # Filter distance_pvals based on the interesting interactions from comb_filt
  distance_pvals_interesting2 <- distance_pvals[distance_pvals$interaction %in% comb_filt$interaction, ]
  
  # Filter distance_pvals_interesting2 for non-NA treatment values
  distance_pvals_interesting2 <- distance_pvals_interesting2 %>% filter(!is.na(treatment))
} else {
  # Filter distance_pvals_sig for non-NA interaction and logfold_group values
  distance_pvals_sig <- distance_pvals_sig %>%
    filter(!is.na(interaction)) %>%
    filter(!is.na(logfold_group))
  
  # Calculate absolute logfold values
  distance_pvals_sig$abs_logfold <- abs(distance_pvals_sig$logfold_group)
  
  # Filter distance_pvals_sig based on absolute logfold threshold
  distance_pvals_sig_filt <- distance_pvals_sig %>% filter(abs_logfold >= 0.1)
  
  # Remove duplicated interactions based on sorted celltype1 and celltype2 columns
  distance_pvals_sig_filt <- distance_pvals_sig_filt[!duplicated(t(apply(distance_pvals_sig_filt[c("celltype1", "celltype2")], 1, sort))), ]
  
  # Filter distance_pvals based on the interesting interactions from distance_pvals_sig_filt
  distance_pvals_interesting <- distance_pvals[distance_pvals$interaction %in% distance_pvals_sig_filt$interaction, ]
}


##### Dumbell plot
select a list of interactions (either as manual vector or as column of a list)

In [ ]:
if (compare_condition_column != NA) {
  # Pairs to plot in Dumbell plot
  pair_to <- unique(distance_pvals_interesting2$interaction)
  
  # Colors used in Dumbell plot
colors <- c("#00BFC4", "#F8766D")

# Dumbbell plot
data <- distance_pvals %>%
  dplyr::filter(!is.na(interaction))

distance_pvals$pairs <- paste0(distance_pvals$celltype1, "_", distance_pvals$celltype2)
distance_pvals_sub <- distance_pvals[distance_pvals$interaction %in% pair_to, ]

distance_pvals_sub <- distance_pvals_sub %>% filter(!is.na(treatment)) %>% arrange(logfold_group) %>% mutate(interaction = factor(interaction, unique(interaction)))
#distance_pvals_sub$interaction <- factor(distance_pvals_sub$interaction, levels=unique(distance_pvals_sub$interaction))

ggplot2::ggplot(data = distance_pvals_sub %>%
  dplyr::filter(!is.na(interaction))) +
  ggplot2::geom_vline(mapping = ggplot2::aes(xintercept = 0), linetype = "dashed") +
  ggplot2::geom_line(
    mapping = ggplot2::aes(x = logfold_group, y = interaction),
    na.rm = TRUE
  ) +
  ggplot2::geom_point(
    mapping = aes(x = logfold_group, y = interaction, fill = treatment, shape = treatment),
    size = 4, stroke = 0.5, na.rm = TRUE
  ) +
  ggplot2::scale_shape_manual(values = c(24, 22)) +
  ggplot2::scale_fill_manual(values = colors) +
  ggplot2::theme_bw() +
  ggplot2::theme(
    panel.grid.major.x = element_blank(),
    panel.grid.minor.x = element_blank(),
    axis.text.y = element_text(size = 16),
    axis.text.x = element_text(size = 16, angle = 45, hjust = 1),
    axis.title.y = element_text(size = 16),
    axis.title.x = element_text(size = 16)
  )

ggsave(paste0(output_path, "dumbbell.pdf"))
} 

## Visualization as igraph

In [ ]:
if (compare_condition_column != NA) {
distance_pvals_sub_grouped <- distance_pvals_sub %>% group_by(celltype1, celltype2)

pairs <- unique(distance_pvals_sub$pairs)

result_list <- list()

for (p in pairs) {
  distance_pvals_sub_filt_neg <- distance_pvals_sub %>% filter(pairs == p) %>% filter(treatment == condition_2)
  distance_pvals_sub_filt_pos <- distance_pvals_sub %>% filter(pairs == p) %>% filter(treatment == condition_1)
  
  difference <- abs(distance_pvals_sub_filt_neg$logfold_group - distance_pvals_sub_filt_pos$logfold_group)
  
  if(0 > (distance_pvals_sub_filt_neg$logfold_group - distance_pvals_sub_filt_pos$logfold_group)){
    direction <- "#3976AC"
  } else {
    direction <- "#C63D30"
  }
  
  df_res <- data.frame (celltype1  = distance_pvals_sub_filt_neg$celltype1,
                    celltype2  = distance_pvals_sub_filt_neg$celltype2,
                    difference = difference,
                    direction = direction
  )
  
  result_list[[p]] <- df_res
}

graph_df <- as.data.frame(do.call(rbind, result_list))

mat <- graph_df

cci_control <- mat



g <- graph_from_data_frame(data.frame(cci_control))
E(g)$weights <- ifelse(cci_control$difference == 0,
                       1e-10, abs(cci_control$difference))

h4 <- hcl.colors(n = 10, palette = "Dynamic")
V(g)$color <- h4
# pdf("figures/WilkEtAl/cellchat_CCI_network_byCondition_Control_network.pdf",
#     width = 8,
#     height = 6)


radian.rescale <- function(x, start=0, direction=1) {
  c.rotate <- function(x) (x + start) %% (2 * pi) * direction
  c.rotate(scales::rescale(x, c(0, 2 * pi), range(x)))
}

lab.locs <- radian.rescale(x=1:10, direction=-1, start=0)

plot(g, 
     vertex.size = 17,
     vertex.color = V(g)$color,
     vertex.label.color = "black",
     vertex.label.cex = 1,
     vertex.label.dist=4,
     vertex.label.degree=lab.locs,
     edge.width = E(g)$weights * 20,
     edge.arrow.size = log(1/E(g)$weights)/80,
     edge.color = E(g)$direction,
     edge.curved = 0.1,
     asp = 0.9,
     layout = layout_in_circle,
     main = compare_condition_column)
# dev.off()
} 

# All interactions 

In [ ]:
if (compare_condition_column != NA) {

pairs <- unique(distance_pvals_sig$interaction)

distance_pvals_sub2 <- distance_pvals[distance_pvals$interaction %in% pairs, ]


distance_pvals_sub_grouped <- distance_pvals_sub2 %>% group_by(celltype1, celltype2)



result_list <- list()

for (p in pairs) {
  distance_pvals_sub_filt_neg <- distance_pvals_sub2 %>% filter(interaction == p) %>% filter(treatment == condition_2)
  distance_pvals_sub_filt_pos <- distance_pvals_sub2 %>% filter(interaction == p) %>% filter(treatment == condition_1)
  
  difference <- abs(distance_pvals_sub_filt_neg$logfold_group - distance_pvals_sub_filt_pos$logfold_group)
  
  if(0 > (distance_pvals_sub_filt_neg$logfold_group - distance_pvals_sub_filt_pos$logfold_group)){
    direction <- "#3976AC"
  } else {
    direction <- "#C63D30"
  }
  
  df_res <- data.frame (celltype1  = distance_pvals_sub_filt_neg$celltype1,
                    celltype2  = distance_pvals_sub_filt_neg$celltype2,
                    difference = difference,
                    direction = direction
  )
  
  result_list[[p]] <- df_res
}

graph_df <- as.data.frame(do.call(rbind, result_list))



graph_df <- graph_df[!duplicated(t(apply(graph_df[c("celltype1", "celltype2")], 1, sort))), ]

mat <- graph_df

cci_control <- mat


g <- graph_from_data_frame(data.frame(cci_control))
E(g)$weights <- ifelse(cci_control$difference == 0,
                       1e-10, abs(cci_control$difference))

h4 <- hcl.colors(n = 18, palette = "Dynamic")
V(g)$color <- h4
# pdf("figures/WilkEtAl/cellchat_CCI_network_byCondition_Control_network.pdf",
#     width = 8,
#     height = 6)


radian.rescale <- function(x, start=0, direction=1) {
  c.rotate <- function(x) (x + start) %% (2 * pi) * direction
  c.rotate(scales::rescale(x, c(0, 2 * pi), range(x)))
}

lab.locs <- radian.rescale(x=1:18, direction=-1, start=0)

plot(g, 
     vertex.size = 17,
     vertex.color = V(g)$color,
     vertex.label.color = "black",
     vertex.label.cex = 1,
     vertex.label.dist=2.5,
     vertex.label.degree=lab.locs,
     edge.width = E(g)$weights * 25,
     edge.arrow.size = log(1/E(g)$weights)/80,
     edge.color = E(g)$direction,
     edge.curved = 0,
     asp = 0.9,
     layout = layout_in_circle,
     main = compare_condition_column)
# dev.off()
} 

# Interactions within a group

In [ ]:
# Get unique interaction pairs
pairs <- unique(distance_pvals_sig$interaction)

# Filter distance_pvals based on the selected interaction pairs
distance_pvals_sub2 <- distance_pvals[distance_pvals$interaction %in% pairs, ]

# Group distance_pvals_sub2 by celltype1 and celltype2
distance_pvals_sub_grouped <- distance_pvals_sub2 %>% group_by(celltype1, celltype2)

# Initialize result lists
result_list1 <- list()
result_list2 <- list()

# Iterate over each pair
for (p in pairs) {
  # Filter distance_pvals_sub2 for negative treatment in the interaction
  distance_pvals_sub_filt_neg <- distance_pvals_sub2 %>% filter(interaction == p) %>% filter(treatment == condition_2)
  
  # Filter distance_pvals_sub2 for positive treatment in the interaction
  distance_pvals_sub_filt_pos <- distance_pvals_sub2 %>% filter(interaction == p) %>% filter(treatment == condition_1)
  
  # Determine the direction color based on logfold_group value
  if (0 > distance_pvals_sub_filt_neg$logfold_group) {
    direction <- "#3976AC"  # Blue color for negative direction
  } else {
    direction <- "#C63D30"  # Red color for positive direction
  }
  
  # Create a data frame for negative treatment
  df_res2 <- data.frame (celltype1  = distance_pvals_sub_filt_neg$celltype1,
                    celltype2  = distance_pvals_sub_filt_neg$celltype2,
                    logfold = distance_pvals_sub_filt_neg$logfold_group,
                    direction = direction,
                    group = condition_2
  )
  
  # Determine the direction color based on logfold_group value
  if (0 > distance_pvals_sub_filt_pos$logfold_group) {
    direction <- "#3976AC"  # Blue color for negative direction
  } else {
    direction <- "#C63D30"  # Red color for positive direction
  }
  
  # Create a data frame for positive treatment
  df_res1 <- data.frame (celltype1  = distance_pvals_sub_filt_pos$celltype1,
                    celltype2  = distance_pvals_sub_filt_pos$celltype2,
                    logfold = distance_pvals_sub_filt_pos$logfold_group,
                    direction = direction,
                    group = condition_1
  )
  
  # Store the data frames in result lists
  result_list1[[p]] <- df_res1
  result_list2[[p]] <- df_res2
}

# Convert result lists to data frames
graph_df1 <- as.data.frame(do.call(rbind, result_list1))
graph_df2 <- as.data.frame(do.call(rbind, result_list2))

# Create a list of graph data frames
graph_list <- list(graph_df1, graph_df2)

# Iterate over each graph data frame
for (x in 1:2) {
  graph_df <- graph_list[[x]]
  
  # Remove duplicated edges
  graph_df <- graph_df[!duplicated(t(apply(graph_df[c("celltype1", "celltype2")], 1, sort))), ]
  
  # Create a graph object
  mat <- graph_df
  cci_control <- mat
  g <- graph_from_data_frame(data.frame(cci_control))
  
  # Set edge weights based on logfold values
  E(g)$weights <- ifelse(cci_control$logfold == 0,
                       1e-10, abs(cci_control$logfold))
  
  # Set vertex colors
  h4 <- hcl.colors(n = 18, palette = "Dynamic")
  V(g)$color <- h4
  
  # Plot the graph
  radian.rescale <- function(x, start=0, direction=1) {
    c.rotate <- function(x) (x + start) %% (2 * pi) * direction
    c.rotate(scales::rescale(x, c(0, 2 * pi), range(x)))
  }
  
  lab.locs <- radian.rescale(x=1:18, direction=-1, start=0)
  
  plot(g, 
       vertex.size = 17,
       vertex.color = V(g)$color,
       vertex.label.color = "black",
       vertex.label.cex = 1,
       vertex.label.dist=2.5,
       vertex.label.degree=lab.locs,
       edge.width = E(g)$weights * 20,
       edge.arrow.size = log(1/E(g)$weights)/80,
       edge.color = E(g)$direction,
       edge.curved = 0,
       asp = 0.9,
       layout = layout_in_circle,
       main = paste0(compare_condition_column, "_", unique(graph_df$group)))
}

